In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, MaxPool2D, Conv2D, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from keras.utils.np_utils import to_categorical
import tensorflow as tf

In [ ]:
train =pd.read_csv('../input/digit-recognizer/train.csv')
test =pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:

print(train.columns)
print('shape of train data is ', train.shape)

In [ ]:
train.sample(5)

EDA and preprocessing

In [ ]:
y_train = train['label']
X_train = train.drop('label', axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
#normalization
X_train = X_train / 255.0
test = test / 255.0

#reshape
X_train = X_train.values.reshape(-1, 28,28,1)
test = test.values.reshape(-1, 28,28,1)

#label encoding
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)
print("X_train shape: ", X_train.shape,"\nX_test shape: ", X_test.shape,
      "\ny_train shape: ", y_train.shape,"\ny_test shape: ", y_test.shape)

In [ ]:
for i in range(1,5):
    p = plt.imshow(X_train[i])
    plt.title(f'Label: {y_train[i]}')
    plt.show()


**MODEL**

In [ ]:
# hyperparameters
IMG_SIZE = 28
batch_size= 86
AUTOTUNE = tf.data.experimental.AUTOTUNE
num_classes = 10

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, MaxPool2D, Conv2D, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers

In [ ]:
def create_model(train_ds, val_ds, epochs):
    # training the model
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape=(28,28,1)),
        layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
        layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.2),
        layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
        layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(512, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(256, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.35),
    
        layers.Dense(10, activation = 'softmax')
        
    ])
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
    
    print(model.summary())
    
    return model

In [ ]:
cnn_model = create_model(X_train, y_train, 100)

In [ ]:

optimizer = RMSprop(lr = 0.001, rho = 0.9, epsilon=1e-08, decay =0.0)
cnn_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
#Data Augmentation
datagen = ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=None,
    preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
)


datagen.fit(X_train)

In [ ]:
# Fit the model
history = cnn_model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = 28, validation_data = (X_test,y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
prediction = cnn_model.predict(X_test).argmax(axis=1)


In [ ]:
score = cnn_model.evaluate(X_test, y_test, verbose = 0)
print("Test Accuracy: ",score[1]) 

In [ ]:
result = cnn_model.predict(test)

result = np.argmax(result, axis=1)

result = pd.Series(result, name='Label')
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),result],axis = 1)

submission.to_csv("submission.csv",index=False)

In [ ]:
x =pd.read_csv("submission.csv")
x.head()